# Express.js Middleware

## Introduction to Middleware
Middleware functions in Express.js are functions that process HTTP requests before they reach route handlers. They have access to the **request object (req)**, **response object (res)**, and the **next function** to pass control to the next middleware or route handler. Middleware sits between incoming requests and final responses, enabling tasks like logging, authentication, and data modification during the request-response cycle.[1][2][3]

Key capabilities include:
- Executing arbitrary code.
- Modifying req and res objects.
- Ending the request-response cycle with res.send().
- Calling next() to proceed to the next middleware.[4]

Without calling next(), requests hang indefinitely, causing browser timeouts.[1]

## Core Concepts and Request Flow
In Express, requests flow through a stack of middleware functions in the order they are defined. A basic request cycle matches routes like app.get('/', ...) after middleware processes the req.[2]

**Request-Response Cycle Visualization**:
- Request arrives → Middleware 1 (modifies req, calls next()) → Middleware 2 → Route handler → Response sent.

**next() Function**: Essential callback that hands control to the subsequent middleware or handler. Omitting it stalls the request.[5]

Order matters: Earlier middleware changes propagate downstream, overwriting prior modifications (e.g., req.harry = '56' in first middleware, overridden in second).[1]

## Writing and Using Middleware
Create middleware as functions with signature `(req, res, next)`. Load via app.use() for application-wide execution.[2]

**Basic Syntax**:
```javascript
app.use((req, res, next) => {
  console.log('Middleware executed');
  next();  // Proceed
});
```

**Examples**:
- **Logging Middleware**: Adds timestamp and method.
  ```javascript
  app.use((req, res, next) => {
    console.log(`${Date.now()} - ${req.method} ${req.url}`);
    fs.appendFileSync('logs.txt', `GET request at ${Date.now()}\n`);
    next();
  });
  ```
- **Modifying Request**: Attach custom properties.
  ```javascript
  app.use((req, res, next) => {
    req.harry = 56;
    console.log(req.headers);
    next();
  });
  ```
  Access in routes: res.send(`Hello ${req.harry}`).[1]

**Ending Response Early**: Use res.send() instead of next() to terminate (e.g., "Hacked by middleware!"). Calling next() after res.send() causes "Cannot set headers after they are sent" errors.[2][1]

**Mount Paths**: Limit scope, e.g., app.use('/user/:id', middleware) executes only for matching paths.[2]

## Types of Middleware
Express supports five middleware types, each with specific use cases.[6][2]

| Type | Description | Example |
|------|-------------|---------|
| **Application-level** | Applies to entire app via app.use(). Ideal for global tasks like logging. | `app.use(express.json())` parses JSON bodies [7]. |
| **Router-level** | Bound to express.Router() instances for route groups. | `router.use((req, res, next) => { ... }); app.use('/blog', router);` [2]. |
| **Error-handling** | Signature `(err, req, res, next)`. Catches errors globally. | `app.use((err, req, res, next) => { res.status(500).send('Error!'); });` [2]. |
| **Built-in** | Native Express functions like express.static (serves files), express.json(), express.urlencoded() [7][1]. | `app.use(express.static('public'));` serves harry.txt.gz. |
| **Third-party** | NPM packages like cookie-parser. | `npm i cookie-parser; app.use(cookieParser());` [2]. |

**Sub-stacks**: Chain multiple via app.use(fn1, fn2) or arrays [logStuff] for reusable groups.[2]

**Route Skipping**: Use next('route') to bypass remaining middleware in a stack.[2]

## Practical Applications and Best Practices
- **Logging**: Track timestamps, methods, URLs; write to files for analysis.[1]
- **Authentication**: Check req.headers['x-auth'] before routes; return 401 if invalid.[2]
- **Static Files**: app.use(express.static('public')) serves assets efficiently.[1]
- **Order Sensitivity**: Define logging before parsing, authentication before sensitive routes.[5]
- **Async Middleware**: Express 5+ handles Promise rejections by calling next(err) automatically.[4]

Common Pitfalls:
- Forgetting next() → Hanging requests.
- res.send() + next() → Header errors.
- Incorrect error middleware signature → Fails silently.[2]

## Summary
Express middleware powers request processing through a flexible stack system, enabling logging, modification, and control flow. Master application/router-level types, always call next(), and leverage built-in/third-party options for robust apps. Practice by building loggers and auth checkers to solidify understanding.